# Ejercicio EvCont4 (a) - Compresión LZZ7

* Óscar Sementé Solà
* Abdelkarim Azzouguagh Ouniri
* Rodrigo Cabezas Quirós

In [47]:
import math, re

class LZZ7Compressor:
    
    def __init__(self, win, wsld):
        self.win = win
        self.wsld = wsld
        self.win_b = int(math.log2(win))
        self.wsld_b = int(math.log2(wsld))
    
    def __is_power_of_two(self, v):
        return math.log2(v).is_integer()
    
    def __ld_to_bin(self, l, d, useLD):
        """
        Transforma (L, D) a formato binario de longitud fija.
        """
        b_win = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.win_b)]))
        b_wsld = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(self.wsld_b)]))
        if useLD:
            return (l, d)
        return b_win(l) + b_wsld(d)
    
    def __match_pattern(self, slide, inp, useLD):
        """
        Busca un patrón de la ventana de entrada en la ventana deslizante. Si lo encuentra devuelve
        (L, D) en decimal o en formato binario (según valor de 'useLD') y la longitud del patrón
        encontrado. Si no lo encuentra devuelve False y 1 (incremento del offset).
        """
        occ, seqs = [], [inp[:i] for i in range(len(inp), 0, -1)]
        while seqs:
            current = seqs.pop(0)
            occ = [(x.start(0), x.end(0)) for x in re.finditer(current, slide)]
            if occ:
                si, ei = occ[-1]
                return self.__ld_to_bin(len(slide[si:ei]), len(slide) - si, useLD), ei - si
        return False, 1
    
    def compress(self, m, useLD=False):
        """
        Compresión de una string binaria a partir de la configuración del compresor.
        """
        if not self.win <= self.wsld and not len(m) >= self.win + self.wsld:
            """
            Que los bits de conf. sean potencia de 2 no tiene ningún sentido. El ejercicio evaluado en
            clase tenía por parametros 8 y 6 bits. Aplicando la norma de la potencia de 2, con 6 bits no
            funcionaria el algoritmo. Así que hemos aplicado esta política.
            or \ not self.__is_power_of_two(self.win) \ or not self.__is_power_of_two(self.wsld):
            """
            return -1
        # Se aplica método de inserción de bit, para los casos problematicos.
        offset, m = 0, self.__add_insertion_bits(m)
        ret = [m[:self.wsld]]
        while offset + self.wsld + self.win <= len(m):
            # Búsqueda de patrón de la ventana de entrada en la deslizante.
            slide = m[offset:offset + self.wsld]
            inp = m[offset + self.wsld:offset + self.wsld + self.win]
            found, flen = self.__match_pattern(slide, inp, useLD)
            if found:
                # Si se encuentra se añade al retorno.
                ret.append(found)
            else:
                # Si no se guarda el símbolo (en formato (L, D) tal y como especifica el enunciado).
                ret.append(self.__ld_to_bin(1, 1, useLD))
            offset -= -flen
        # Si al acabar quedan bits fuera de las ventanas, se añaden al final de la cadena.
        if offset < len(m):
            ret.append(m[offset + self.wsld:])
        if useLD:
            return ret
        return "".join(ret)
    
    def uncompress(self, m):
        """
        Descompresión de una string binaria a partir de la configuración del compresor.
        """
        ret = m[:self.wsld]
        div = [m[i:i+self.wsld_b+self.win_b] for i in range(self.wsld, len(m), self.wsld_b + self.win_b)]
        lf = lambda x: 2**self.win_b if x == 0 else x
        df = lambda x: 2**self.wsld_b if x == 0 else x
        while div:
            c = div.pop(0)
            if len(c) < self.wsld_b + self.win_b:
                ret = ret + c
            else:
                l, d = lf(int(c[:self.win_b], 2)), df(int(c[self.win_b:], 2))
                ret = ret + ret[len(ret) - d: len(ret) + l - d]
        return self.__remove_insertion_bits(ret)
    
    def __add_insertion_bits(self, m):
        """
        Aplica método de inserción de bit para las entradas con un solo carácter.
        """
        asd = [m[i:i + self.wsld - 1] for i in range(0, len(m), self.wsld - 1)]
        opposite = lambda x: "0" if x == "1" else "1"
        for i in range(len(asd)):
            if asd[i] == asd[i][0] * len(asd[i]) and len(asd[i]) == self.wsld - 1:
                asd[i] = asd[i] + opposite(asd[i][0])
        return "".join(asd)
    
    def __remove_insertion_bits(self, m):
        """
        Desaplica método de inserción de bit para las entradas con un solo carácter.
        """
        opposite = lambda x: "0" if x == "1" else "1"
        for i in range(0, len(m), self.wsld):
            p = m[i:i + self.wsld]
            if p[:-1] == p[0] * len(p[:-1]) and p[-1] == opposite(p[0]):
                m = m[:i + self.wsld-1] + m[i+self.wsld:]
        return m

In [48]:
def test(msg, comp, i=False):
    if i != False: print("Test #{}".format(i))
    print("-> Original:\t\t\t", msg)
    print("-> Compresión:\n\tCon (L, D):\t\t", comp.compress(msg, useLD=True))
    asd = comp.compress(msg)
    print("\tBinario:\t\t", asd)
    asd2 = comp.uncompress(asd)
    print("-> Descompresión:\t\t", asd2)
    # Comprobación con el mensaje original.
    assert msg == asd2

msg = "11011100101001111010100010001"
comp = LZZ7Compressor(6, 8)
test(msg, comp)

-> Original:			 11011100101001111010100010001
-> Compresión:
	Con (L, D):		 ['11011100', (3, 7), (3, 5), (1, 1), (2, 2), (3, 8), (2, 2), (1, 1), (4, 4), '01']
	Binario:		 11011100111111110101001100101100010010010010010001
-> Descompresión:		 11011100101001111010100010001


#### [Apartado 1] Comprobad que el programa comprime y descomprime correctamente una cadena de 25 bits aleatorios con Mdes = 8 y Ment = 4. Ayuda: Podéis generar bits aleatorios con Math.round(Math.random()).

In [23]:
import random
def rand_key(p):
    key1 = ""
    for i in range(p):
        temp = str(random.randint(0, 1))
        key1 += temp 
    return(key1)

In [53]:
comp = LZZ7Compressor(4, 8)
test(rand_key(25), comp, i)

Test #4
-> Original:			 0110010001111100110000111
-> Compresión:
	Con (L, D):		 ['01100100', (3, 8), (2, 2), (3, 8), (4, 4), (4, 6), '1']
	Binario:		 0110010011000100101100000100001101
-> Descompresión:		 0110010001111100110000111


#### [Apartado 2] Utilizad el programa anterior para determinar si es posible, ajustando los valores de Mdes y Ment, conseguir comprimir datos aleatorios mediante LZ77 (es decir, que la cadena de datos originales sea más larga que la cadena comprimida). ¿Por qué? ¿Cuál es la máxima compresión que lográis? ¿Con qué valores? (Ayuda: utilizad una cadena de datos de entrada de, por lo menos, 10000 bits aleatorios. Ajustad Mdes y Ment entre 2 y 2048).

In [54]:
def test2(msg, comp, i=False):
    asd = comp.compress(msg)
    asd2 = comp.uncompress(asd)
    if len(msg) > len(asd):
        if i != False: print("Test #{}".format(i))
        print("-> Original:\t\t\t", msg)
        print("-> Compresión:\n\tCon (L, D):\t\t", comp.compress(msg, useLD=True))
        print("\tBinario:\t\t", asd)
        print("-> Descompresión:\t\t", asd2)
        print("-> Longitud original:\t\t{}", len(msg))
        print("-> Longitud comprimida:\t\t{}", len(asd))
        # Comprobación con el mensaje original.
        assert msg == asd2

In [ ]:
asd = [(2, 2048)]
for i in range(len(asd)):
    msld, mwin = asd[i]
    test2(rand_key(1000), LZZ7Compressor(mwin, msld), i)